In [ ]:
import os
import warnings
import pathlib
from glob import glob
import time
import csv
import traceback
import shutil
import psutil

import earthpy as et
import earthpy.appeears as etapp
import earthpy.earthexplorer as etee
import earthpy.spatial as es
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import numpy as np
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import xarray as xr
import xrspatial
import pandas as pd
import folium
import cartopy.crs as ccrs
from zipfile import BadZipFile
from pathlib import Path
import requests

utm_zone = 32614

In [ ]:
def create_data_directory(name):
    """
    Create a data directory for data.

    This function takes a `name` parameter and creates a directory structure
    where NAIP data can be stored. The directory is created inside the
    'earth-analytics' directory in the user's home folder.

    Parameters:
    - name (str): The name of the directory to be created.

    Returns:
    - str: The path to the created data directory.
    """

    # Create a variable that stores the path to a directory.
    data_path = os.path.join(
        pathlib.Path.home(), 'earth-analytics', 'data', 'final', name)

    # Create the directory if it doesn't already exist.
    os.makedirs(data_path, exist_ok=True)

    # Print the path to the data directory.
    print("Data Directory:", data_path)

    # Return the path to the data directory.
    return data_path

In [ ]:
# Set data sub-directory name
set_directory_name = "usa_grassland"

# Run create_data_directory function
data_path = create_data_directory(set_directory_name)
data_path

In [ ]:
def check_shapefile_to_gdf(gdf_name, shapefile_url, column_name):
    """
    Check if a GeoDataFrame with a specified name exists. If it exists,
    return it; otherwise, download a shapefile from a given URL,
    create a GeoDataFrame, and assign it to the specified name.

    Parameters:
    - gdf_name (str): The name of the GeoDataFrame variable to check/create.
    - shapefile_url (str): The URL to download the shapefile.
    - column_name (str): The column name to set as the index in the GeoDataFrame.

    Returns:
    - gpd.GeoDataFrame: The existing or newly created GeoDataFrame.
    """
    try:
        # Attempt to get the GeoDataFrame from the global scope
        gdf = globals()[gdf_name]

        # If the variable exists, print a message and return it
        print(f"{gdf_name} already exists.")
        return gdf

    except KeyError:
        # If the variable does not exist, create it
        print(f"{gdf_name} does not exist yet.")

        # Shapefile Location URL
        print("Downloading Shapefile:")

        # Create GeoDataFrame from the shapefile
        gdf = gpd.read_file(shapefile_url)
        # Set the specified column as the index
        gdf = (
            gdf.set_index(column_name)
            .to_crs(utm_zone)
        )
        # Assign the GeoDataFrame to the specified name in the global scope
        globals()[gdf_name] = gdf

        print("Done. Shapefile to Geodataframe.")

        # Return the GeoDataFrame
        return gdf

# Set up check_shapefile_to_gdf function
all_grasslands_url = (
    "https://data.fs.usda.gov/geodata/edw/edw_resources/shp/S_USA.NationalGrassland.zip")                        

# Specify the variable name to check, in quotes
all_grasslands_gdf_name = "usa_grasslands_gdf"

#Specify column name of locations
gdf_grassland_name_column = 'GRASSLANDN'

# Run check_shapefile_to_gdf function
all_grasslands_gdf = check_shapefile_to_gdf(
    all_grasslands_gdf_name, all_grasslands_url,
    gdf_grassland_name_column)

all_grasslands_gdf

print(all_grasslands_gdf.crs)

In [ ]:
def site_map(gdf, title, savename):
    # Create GeoViews Polygons from the GeoDataFrame
    polys = gv.Polygons(gdf.to_crs(4326), vdims=[all_grasslands_gdf.index.name])

    # Choropleth map
    plot = gv.tile_sources.CartoLight() * polys.opts(
        alpha=0.5, fill_alpha=0.7, tools=['hover'],
        hover_fill_alpha=0.9, hover_fill_color='red',
        title=title, aspect=1
    )

    # Set plot dimensions
    plot.opts(width=500)

    # Save the choropleth map as an HTML file
    hv.save(plot, f'{savename}.html')
    return plot

In [ ]:
site_map_gdf = all_grasslands_gdf
plot_title = 'US National Grasslands'
save_plots_as = 'all-grasslands'

plot = site_map(site_map_gdf, plot_title, save_plots_as)
plot

In [ ]:
# Create GeoViews Polygons from the GeoDataFrame
texas_grasslands_gdf = all_grasslands_gdf.loc[[(
    'Lyndon B. Johnson National Grassland')]]

print(texas_grasslands_gdf)

site_map_gdf = texas_grasslands_gdf
plot_title = 'Lyndon B. Johnson National Grassland, Texas, USA.'
save_plots_as = 'tx-grasslands'

plot = site_map(site_map_gdf, plot_title, save_plots_as)
plot

In [ ]:
# Create GeoViews Polygons from the GeoDataFrame
nd_grasslands_gdf = all_grasslands_gdf.loc[[(
    'Sheyenne National Grassland')]]
print(nd_grasslands_gdf)

site_map_gdf = nd_grasslands_gdf
plot_title = 'Sheyenne National Grassland, North Dakota, USA.'
save_plots_as = 'nd-grasslands'

plot = site_map(site_map_gdf, plot_title, save_plots_as)
plot

In [ ]:
def polaris_to_da(directory_name, file_names, gdf, plot_title):
    # Base URL where the .tif files are hosted
    base_url = 'http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/ph/mean/100_200/'

    # Run create_data_directory function
    data_path = create_data_directory(directory_name)
    # Ensure the output directory exists
    output_directory = Path(data_path)

    # List to store downloaded file paths
    downloaded_files = []

    # Loop through each file name
    for file_name in file_names:
        # Construct the full URL for the file
        file_url = f'{base_url}{file_name}'

        # Construct the full output path
        if len(file_names) == 1:
            output_path = output_directory / file_name
        else:
            output_path = output_directory / Path(file_name)

        # Download the file
        response = requests.get(file_url)
        if response.status_code == 200:
            # Save the file to the output directory
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f'Downloaded: {output_path}')
            downloaded_files.append(output_path)
        else:
            print(f'Error downloading {file_name}. Status code: {response.status_code}')

    # List to store data arrays
    data_arrays = []

    # Loop through each downloaded file path
    for file_path in downloaded_files:
        # Open the .tif file using rioxarray
        xarr_data = rxr.open_rasterio(file_path).squeeze()

        # Append the xarray DataArray to the list
        data_arrays.append(xarr_data)
        print(f'Opened and read: {file_path}')

    # Merge data arrays into a single dataset
    merged_da = (
        rxr.merge.merge_arrays(data_arrays)
        .rio.reproject(utm_zone)
        .rio.clip_box(*gdf.total_bounds)
    )

    # Plot
    plot = merged_da.hvplot(x='x',y='y',rasterize=True).opts(
        tools=['hover'], title=plot_title, aspect=1
    )


    # Save the choropleth map as an HTML file
    hv.save(plot, f'{directory_name}.html')

    return plot, merged_da


Download model variables as raster layers covering your study area envelope, including:

    Elevation from the SRTM (available from the APPEEARS API)

Note: There are two download_elevation functions, the API never works well so the first one will attempt to download the data, and then if fails will manually upload .tif files from the same directory.

If you only want to use the manually downloaded data, then the second function should be used. Otherwise, use the first function, and the second as a backup.


In [ ]:
# def download_elevation(directory_name, gdf, other_da, plot_title):
#     from requests.exceptions import HTTPError

#     try:
#         # Run create_data_directory function
#         data_path = create_data_directory(directory_name)
#         # Define the parameters

#         downloader = etapp.AppeearsDownloader(
#             download_key=(directory_name),
#             ea_dir=data_path,
#             product='SRTMGL1_NC.003',
#             layer='SRTMGL1_DEM',
#             start_date='02-01-2000',
#             end_date='02-21-2000',
#             polygon=gdf
#         )
#         print("Downloading Appears SRTM Data...")

#         downloader.download_files()    
        
#         print(f"Appears Data Downloaded: {data_path}")
        
#         # Extract Filenames
#         path_list = glob(os.path.join(downloader.data_dir, '*', '*DEM*.tif'))
#         print(path_list)
        
#         # List to store data arrays
#         data_arrays = []

#         # Loop through each downloaded file path
#         for file_path in path_list:
#             # Open the .tif file using rioxarray
#             xarr_data = rxr.open_rasterio(file_path, masked=True).squeeze()

#             # Append the xarray DataArray to the list
#             data_arrays.append(xarr_data)
#             print(f'Opened and read: {file_path}')

#         # Merge data arrays into a single dataset
#         merged_da = (
#             rxr.merge.merge_arrays(data_arrays)
#             .rio.reproject_match(other_da)
            
#         )

#         plot = xrspatial.aspect(merged_da).hvplot(x='x', y='y', title=plot_title, colormap='colorwheel')

#         return plot, merged_da

#     except HTTPError as e:
#         if e.response.status_code == 500:
#             print("\nAPPEARS Internal Server Issue, uploading manually downloaded data.")
            
#             # Open the .tif file using rioxarray

#             print(str(directory_name))
#             if str(directory_name).startswith("tx"):
#                 xarr_data = rxr.open_rasterio("tx_srtm_data.tif", masked=True).squeeze()
#                 print(f'Opened and read: Texas SRTM Data')

#             else:
#                 xarr_data = rxr.open_rasterio("nd_srtm_data.tif", masked=True).squeeze()
#                 print(f'Opened and read: North Dakota SRTM Data')

#             xarr_data = xarr_data.rio.reproject(utm_zone).rio.clip_box(*gdf.total_bounds)

#             plot = xrspatial.aspect(xarr_data).hvplot(x='x', y='y', title=plot_title, colormap='colorwheel')

#             return plot, xarr_data
#         else:
#             print(e.response.status_code)


#     except Exception as e:
#         print(f"An error occurred: {e}. Try again.")

In [ ]:
def download_elevation(directory_name, gdf, other_da, plot_title):
    from requests.exceptions import HTTPError

    print("\nAPPEARS Internal Server Issue, uploading manually downloaded data.")
    
    # Open the .tif file using rioxarray

    print(str(directory_name))
    if str(directory_name).startswith("tx"):
        xarr_data = rxr.open_rasterio("tx_srtm_data.tif", masked=True).squeeze()
        print(f'Opened and read: Texas SRTM Data')

    else:
        xarr_data = rxr.open_rasterio("nd_srtm_data.tif", masked=True).squeeze()
        print(f'Opened and read: North Dakota SRTM Data')
        
    xarr_data = xarr_data.rio.reproject_match(other_da)
    xarr_data = xarr_data.rio.clip_box(*gdf.total_bounds)

    plot = xrspatial.aspect(xarr_data).hvplot(x='x', y='y',title=plot_title, colormap='colorwheel')
    # Save the plot as an HTML file
    hv.save(plot, f'{directory_name}.html')
    
    return plot, xarr_data

Download model variables as raster layers covering your study area envelope, including:

    At least one climate variable from the MACAv2 dataset, accessible from Climate Toolbox. *Undergraduate students may download a single year and scenario; Graduate students should download at least two)



In [ ]:
def macav2_data_to_da(directory_name, data_bounds, title, gdf, other_da):
    # Run create_data_directory function
    data_path = create_data_directory(directory_name)

    # Define URL to download data from
    macav2_data_url = (
        "http://thredds.northwestknowledge.net:8080/thredds/ncss/"
        "agg_macav2metdata_pr_CCSM4_r6i1p1_historical_1950_2005_CONUS_monthly"
        ".nc?var=precipitation"
        f"&north={str(data_bounds[0])}"
        f"&west={str(data_bounds[1])}"
        f"&east={str(data_bounds[2])}"
        f"&south={str(data_bounds[3])}"
        "&horizStride=1&"
        "time_start=2003-12-15T00%3A00%3A00Z&"
        "time_end=2005-12-15T00%3A00%3A00Z&"
        "timeStride=1&accept=netcdf"
    )

    # Get netCDF data
    macav2_data = requests.get(macav2_data_url)

    # Write netCDF data to file
    with open('macav2_data.nc', 'wb') as macav2_data_file:
        macav2_data_file.write(macav2_data.content)

    # Read netCDF data into xarray dataset
    data_array = xr.open_dataset('macav2_data.nc').squeeze()

    # Correct coordinates: Does not work on a global dataset
    data_array = data_array.assign_coords(lon=data_array.lon-360)

    # Set the CRS and grab precipitation
    precip_da = data_array.precipitation.mean('time')
    precip_da = precip_da.rio.write_crs("epsg:4326", inplace=True)
    precip_da = precip_da.rio.set_spatial_dims('lon', 'lat', inplace=True)
    precip_da = precip_da.rio.reproject_match(other_da)
    precip_da = precip_da.rio.clip_box(*gdf.total_bounds)
    # Plot
    plot = precip_da.hvplot(rasterize=True).opts(
        tools=['hover'], title=title, aspect=1
    )

    # Save the plot as an HTML file
    hv.save(plot, f'{directory_name}.html')

    return plot, precip_da

In [ ]:
### Lyndon B. Johnson National Grassland

# Download Polaris Data
tif_files_to_download = ['lat3334_lon-98-97.tif']
directory_save_name = 'tx_polaris_pH'
tx_polaris_plot_title = 'Lyndon B. Johnson National Grassland pH range'

tx_polaris_plot, tx_polaris_da = polaris_to_da(directory_save_name, tif_files_to_download, texas_grasslands_gdf, tx_polaris_plot_title)

# Download AppEARS Data
tx_appears_pathname = 'tx_appears_srtm'
tx_appears_plot_title = 'Lyndon B. Johnson National Grassland\nElevation Data'

tx_appears_plot, tx_appears_da = download_elevation(
    tx_appears_pathname,
    texas_grasslands_gdf,
    tx_polaris_da,
    tx_appears_plot_title)

# Derive Slope from Elevation Data
tx_slope_da = xrspatial.slope(tx_appears_da.rio.reproject_match(tx_polaris_da))
tx_slope_plot = tx_slope_da.hvplot(x='x', y='y', title="Lyndon B. Johnson National Grassland Slope", colormap='viridis_r')

# Download MACAV2 Data
tx_macav2_path = 'macav2_aggregated_monthly_historical'
tx_n_w_e_s_bounds = [33.45, -97.8, -97.45, 33.2]
tx_macav2_title = 'Lyndon B. Johnson National Grassland\nMean Monthly Aggregated Precipitation over Time\n2003 to 2005'

tx_macav2_plot, tx_macav2_da = macav2_data_to_da(tx_macav2_path, tx_n_w_e_s_bounds, tx_macav2_title, texas_grasslands_gdf, tx_polaris_da)

# Combine plots into a single layout
layout = (tx_polaris_plot + tx_appears_plot + tx_slope_plot + tx_macav2_plot).cols(1)

# Save the layout to HTML files
hv.save(layout, 'tx_combined_data_plots.html')
layout

In [ ]:
### Sheyenne National Grassland

# Download Polaris Data
tif_files_to_download = ['lat4647_lon-97-96.tif', 'lat4647_lon-98-97.tif']
directory_save_name = 'nd_polaris_pH'
nd_polaris_plot_title = 'Sheyenne National Grassland pH range'

nd_polaris_plot, nd_polaris_da = polaris_to_da(directory_save_name, tif_files_to_download, nd_grasslands_gdf, nd_polaris_plot_title)

# Download AppEARS Data
nd_appears_pathname = 'nd_appears_srtm'
nd_appears_plot_title = 'Sheyenne National Grassland\nElevation Data'

nd_appears_plot, nd_appears_da = download_elevation(
    nd_appears_pathname,
    nd_grasslands_gdf,
    nd_polaris_da,
    nd_appears_plot_title)

# Derive Slope from Elevation Data
nd_slope_da = xrspatial.slope(nd_appears_da.rio.reproject_match(nd_polaris_da))
nd_slope_plot = nd_slope_da.hvplot(x='x', y='y', title="Sheyenne National Grassland Slope", colormap='viridis_r')

# Download MACAV2 Data
nd_macav2_path = 'macav2_aggregated_monthly_historical'
nd_n_w_e_s_bounds = [46.58, -97.5, -96.9, 46.1]
nd_macav2_title = 'Sheyenne National Grassland\nMean Monthly Aggregated Precipitation over Time\n2003 to 2005'

nd_macav2_plot, nd_macav2_da = macav2_data_to_da(nd_macav2_path, nd_n_w_e_s_bounds, nd_macav2_title, nd_grasslands_gdf, nd_polaris_da)

# Combine plots into a single layout
layout = (nd_polaris_plot + nd_appears_plot + nd_slope_plot + nd_macav2_plot).cols(1)

# Save the layout to HTML files
hv.save(layout, 'nd_combined_data_plots.html')
layout

Calculate at least one derived topographic variable (slope or aspect) to use in your model. You probably will wish to use the xarray-spatial library, which is available in the latest earth-analytics-python environment (but will need to be installed/updated if you are working on your own machine). Note that calculated slope may not be correct if you are using a CRS with units of degrees; you should re-project into a projected coordinate system with units of meters, such as the appropriate UTM Zone.

Harmonize your data - make sure that the grids for each of your layers match up. Check out the ds.rio.reproject_match() method from rioxarray.

In [ ]:
def check_and_set_crs(datasets, reference_dataset):
    updated_datasets = []
    target_crs = reference_dataset.rio.crs if isinstance(
        reference_dataset, xr.DataArray) else reference_dataset.crs
    
    print("Making necessary changes...")

    for ds in datasets:
        if isinstance(ds, xr.DataArray):
            if ds.rio.crs != target_crs:
                ds = ds.rio.set_crs(target_crs)

            # Clip to the bounds of reference_dataset
            if isinstance(reference_dataset, gpd.GeoDataFrame):
                bounds = reference_dataset.total_bounds
                # Create a bounding box geometry from the bounds
                bbox = box(bounds[0], bounds[1], bounds[2], bounds[3])
                # Clip the DataArray using the bounding box
                ds = ds.rio.clip_box(*bbox.bounds)

        elif isinstance(ds, gpd.GeoDataFrame):
            if ds.crs != target_crs:
                ds = ds.to_crs(target_crs)

        else:
            raise ValueError("Unsupported dataset type. Supported types"
                    " are xarray.DataArray and geopandas.GeoDataFrame.")

        updated_datasets.append(ds)

    print("All data is harmonized and clipped.")

    return updated_datasets

In [ ]:
updated_tx_data_arrays = check_and_set_crs([texas_grasslands_gdf, tx_appears_da, tx_macav2_da], tx_polaris_da)

In [ ]:
updated_nd_data_arrays = check_and_set_crs([nd_grasslands_gdf, nd_appears_da, nd_macav2_da], nd_polaris_da)

Research S. nutans, and find out what optimal values are for each variable you are using (e.g. soil pH, slope, and current climatological annual precipitation).

In [ ]:
s_nutans_pH_min = 4.8      # pH low range
s_nutans_pH_max = 8.0      # pH high range

s_nutans_slope_min = 10    # percentage low slope range
s_nutans_slope_max = 40    # percentage low slope range

s_nutans_precip_min = 28   # precip low range
s_nutans_precip_max = 114  # precip low range

For each data in each dataarray, assign a value from 0 to 1 for how close that grid square is to the optimum range (1=optimal, 0=incompatible)

In [ ]:
def binary_da(data_array, optimum_min, optimum_max):
    # Calculate proximity to the optimum range for each grid square
    proximity = 1 - abs((data_array - optimum_min) / (optimum_max - optimum_min))

    # Clip proximity values to ensure they are between 0 and 1
    proximity = proximity.clip(0, 1)

    return proximity

Combine your layers by multiplying them together. This will give you a single suitability number for each square.

In [ ]:
# TX

# Optimum Values
tx_polaris_optimal_da = binary_da(tx_polaris_da, s_nutans_pH_min, s_nutans_pH_max)
tx_slope_optimal_da = binary_da(tx_slope_da, s_nutans_slope_min, s_nutans_slope_max)
tx_macav2_optimal_da = binary_da(tx_macav2_da, s_nutans_precip_min, s_nutans_precip_max)
final_tx_optimal = tx_polaris_optimal_da * tx_slope_optimal_da * tx_macav2_optimal_da

# Create individual plots
tx_pH = tx_polaris_optimal_da.hvplot(x='x', y='y', title="Lyndon B. Johnson National Grassland pH\nOptimized for pH Between 4.8 and 8.0", colormap='viridis_r')
tx_slope = tx_slope_optimal_da.hvplot(x='x', y='y', title="Lyndon B. Johnson National Grassland Slope\nOptimized for Percentage Between 10 and 40", colormap='viridis_r')
tx_precip = tx_macav2_optimal_da.hvplot(title="Lyndon B. Johnson National Grassland Precipitation\nOptimized for Between 28mm and 114mm", colormap='viridis_r')
tx_final = final_tx_optimal.hvplot(x='x', y='y', title="Lyndon B. Johnson National Grassland\nSorghastrum nutans Habitat Suitability Model", colormap='viridis_r')

# Combine plots into a single layout
layout = (tx_pH + tx_slope + tx_precip + tx_final).cols(1)

# Save the layout to HTML files
hv.save(layout, 'tx_combined_optimal_plots.html')
layout

In [ ]:
# ND

# Optimum Values
nd_polaris_optimal_da = binary_da(nd_polaris_da, s_nutans_pH_min, s_nutans_pH_max)
nd_slope_optimal_da = binary_da(nd_slope_da, s_nutans_slope_min, s_nutans_slope_max)
nd_macav2_optimal_da = binary_da(nd_macav2_da, s_nutans_precip_min, s_nutans_precip_max)
final_nd_optimal = nd_polaris_optimal_da * nd_slope_optimal_da * nd_macav2_optimal_da


# Create individual plots
nd_pH = nd_polaris_optimal_da.hvplot(x='x', y='y', title="Sheyenne National Grassland pH\nOptimized for pH Between 4.8 and 8.0", colormap='viridis_r')
nd_slope = nd_slope_optimal_da.hvplot(x='x', y='y', title="Sheyenne National Grassland Slope\nOptimized for Percentage Between 10 and 40", colormap='viridis_r')
nd_precip = nd_macav2_optimal_da.hvplot(title="Sheyenne National Grassland Precipitation\nOptimized for Between 28mm and 114mm", colormap='viridis_r')
nd_final = final_nd_optimal.hvplot(x='x', y='y', title="Sheyenne National Grassland\nSorghastrum nutans Habitat Suitability Model", colormap='viridis_r')

# Combine plots into a single layout
layout = (nd_pH + nd_slope + nd_precip + nd_final).cols(1)

# Save the layout to HTML files
hv.save(layout, 'nd_combined_optimal_plots.html')
layout